In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
device='cuda'

## Download the dataset and preprocess it

In [3]:
!mkdir '/content/Dataset'

In [4]:
!unzip '/content/train.csv.zip' -d '/content/Dataset'
!unzip '/content/test.csv.zip' -d '/content/Dataset'

Archive:  /content/train.csv.zip
  inflating: /content/Dataset/train.csv  
Archive:  /content/test.csv.zip
  inflating: /content/Dataset/test.csv  


In [5]:
!unzip '/content/test.csv.zip' -d '/content/Dataset'

Archive:  /content/test.csv.zip
replace /content/Dataset/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/Dataset/test.csv  


In [6]:
train_df = pd.read_csv('/content/Dataset/train.csv')

In [7]:
train_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
class MNIST_Dataset(Dataset):
  def __init__(self, file_path):
    train_df = pd.read_csv(file_path)
    self.labels = train_df.iloc[:, 0].values
    self.pixels = train_df.iloc[:, 1:].values
    self.pixels = self.pixels.astype(np.float32) / 255.0
    self.pixels = self.pixels.reshape((-1, 1, 28, 28))

  def  __len__(self):
    return self.labels.shape[0]

  def __getitem__(self, index):
    return (self.pixels[index], self.labels[index])

In [9]:
!unzip '/content/test.csv (2).zip' -d '/content/Dataset'

unzip:  cannot find or open /content/test.csv (2).zip, /content/test.csv (2).zip.zip or /content/test.csv (2).zip.ZIP.


In [10]:
full_dataset = MNIST_Dataset('/content/Dataset/train.csv')

train_size = int(0.9 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)

In [11]:
for example in train_dataloader:
  images, labels = example
  print(images.shape)
  print(labels.shape)
  break

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [12]:
test_df = pd.read_csv('/content/Dataset/test.csv')
test_pixels = test_df.iloc[:, :].values
test_pixels = test_pixels.astype(np.float32) / 255.0
test_pixels = test_pixels.reshape((-1, 1, 28, 28))

In [13]:
test_pixels.shape

(28000, 1, 28, 28)

## Build a Model

In [14]:
class SimpleConvNet(nn.Module):
  def __init__(self):
    super(SimpleConvNet, self).__init__()
    self.conv_layer = nn.Sequential(
        # (N, 1, 28, 28)
        nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
        # (N, 32, 28, 28)
        nn.ReLU(),
        # (N, 32, 28, 28)
        nn.MaxPool2d(kernel_size=2, stride=2),
        # (N, 32, 14, 14)
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
        # (N, 64, 14, 14)
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
        # (N, 64, 7, 7)
    )

    self.fc_layer = nn.Sequential(
      # (N, 7*7*64)
      nn.Linear(7*7*64, 1000),
      # (N, 1000)
      nn.ReLU(),
      nn.Linear(1000, 10)
      # (N, 10)
    )


  def forward(self, x):
    '''
    Input:
      x: (N, 1, 28, 28)
    Output:
      pred: (N, 10)
    '''
    x = self.conv_layer(x)
    # (N, 64, 7, 7)
    x = x.view(x.size(0), -1)
    x = self.fc_layer(x)
    return x

In [15]:
example_input = torch.randn(32, 1, 28, 28)
model = SimpleConvNet()
example_output = model(example_input)
print(example_output.shape)

torch.Size([32, 10])


In [16]:
import torch
torch.cuda.empty_cache()

In [17]:
model = SimpleConvNet()
model = model.to(device)

## Train

In [18]:
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(), lr=0.001)

In [19]:
len(train_dataloader)

591

In [20]:
def evalutation():
  model.eval()
  total_loss = 0
  size = len(val_dataloader)
  with torch.no_grad():
    for (images, labels) in val_dataloader:
      images = images.to(device)
      labels = labels.to(device)
      preds = model(images)
      loss = loss_fn(preds, labels)
      total_loss += loss / size

  return total_loss

In [21]:
len(train_dataloader)

591

In [22]:
!mkdir '/content/Checkpoints'

In [23]:
def save_checkpoint(state, filename="checkpoint.pth.tar"):
    torch.save(state, filename)

In [24]:
train_loss_log = []
val_loss_log = []

In [27]:
def train_one_epoch(epoch):
  model.train()
  size = len(train_dataloader)
  total_loss = 0
  for batch_idx, (images, labels) in enumerate(tqdm(train_dataloader)):
    images = images.to(device)
    labels = labels.to(device)
    opt.zero_grad()
    pred = model(images)
    loss = loss_fn(pred, labels)
    total_loss += loss / size
    loss.backward()
    opt.step()

    if batch_idx % 100 == 0:
      val_loss = evalutation()
      print(f"Epoch {epoch} - Batch {batch_idx}|  Train Loss: {loss}, Val Loss: {val_loss}")
      train_loss_log.append(loss)
      val_loss_log.append(val_loss)

      save_checkpoint({
                'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'optimizer' : opt.state_dict(),
            }, filename=f"/content/Checkpoints/checkpoint_epoch_{epoch}_batch_{batch_idx}.pth.tar")

In [28]:
# # prompt: delete all the files at /content/Checkpoints

# !rm -rf /content/Checkpoints/*


In [29]:
for epoch in range(10):
  train_one_epoch(epoch+1)

  4%|▍         | 25/591 [00:00<00:05, 98.11it/s]

Epoch 1 - Batch 0|  Train Loss: 2.3136298656463623, Val Loss: 2.2626832651369497


 22%|██▏       | 130/591 [00:00<00:02, 173.29it/s]

Epoch 1 - Batch 100|  Train Loss: 0.20669490098953247, Val Loss: 0.1938504578376358


 36%|███▌      | 214/591 [00:01<00:02, 156.55it/s]

...

 71%|███████▏  | 422/591 [00:02<00:01, 158.19it/s]

Epoch 10 - Batch 400|  Train Loss: 0.00025150529108941555, Val Loss: 0.053424159816517436


 90%|████████▉ | 530/591 [00:03<00:00, 176.54it/s]

Epoch 10 - Batch 500|  Train Loss: 0.0007673862855881453, Val Loss: 0.04530982657782564


100%|██████████| 591/591 [00:03<00:00, 170.26it/s]


## Eval

In [41]:
checkpoint = torch.load('/content/Checkpoints/checkpoint_epoch_5_batch_400.pth.tar')
model.load_state_dict(checkpoint['state_dict'])
opt.load_state_dict(checkpoint['optimizer'])

In [42]:
test_input = torch.tensor(test_pixels).to(device)

In [45]:
batch_size = 64
def get_batches(data, batch_size):
  for start in range(0, len(data), batch_size):
    end = start + batch_size
    yield data[start:end].to(device)

model.eval()
with torch.no_grad():
  all_preds = []
  for batch in get_batches(test_input, batch_size):
    preds = model(batch)
    all_preds.extend(preds.cpu().numpy())

In [47]:
res = torch.tensor(all_preds)

<ipython-input-47-e6b3488f7189>:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  res = torch.tensor(all_preds)


In [50]:
_, max_indices = torch.max(res, dim=1, keepdim=False)

In [52]:
ImageId = list(range(1, 28001))
max_indices = max_indices.cpu()

In [53]:
df = pd.DataFrame({'ImageId': ImageId, 'Label': max_indices})

In [54]:
df.to_csv('output.csv', index=False)